In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation , Embedding
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
import gensim

from gensim.models import Word2Vec, KeyedVectors

from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import re
import matplotlib.pyplot as plt
import nltk


from tensorflow.keras.utils import to_categorical


In [2]:
data = pd.read_csv("data/news_class.csv")

df = data.copy() # to be safe and avoid errors

df = df.loc[:,["data_id" , "content" , "category_level_1" , "category_level_2"]]

In [3]:
# Function for removing ASCII characters
def _removeNonAscii(s):
    return "".join(i for i in s if  ord(i)<128)

# Function for converting to lower case
def make_lower_case(text):
    return text.lower()

# Function for removing stop words
def remove_stop_words(text):
    text = text.split()
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops]
    text = " ".join(text)
    return text

# Function for removing html
def remove_html(text):
    html_pattern = re.compile('<.*?>')
    return html_pattern.sub(r'', text)

# Function for removing punctuation
def remove_punctuation(text):
    tokenizer = RegexpTokenizer(r'\w+')
    text = tokenizer.tokenize(text)
    text = " ".join(text)
    return text

def lemm_text(text):
    lemm=WordNetLemmatizer()
    tokenizer = RegexpTokenizer(r'\w+')
    text = tokenizer.tokenize(text)
    tokens = text
    return ' '.join([lemm.lemmatize(t) for t in tokens])

In [4]:
df['Cleaned'] = df['content'].apply(_removeNonAscii)
df['Cleaned'] = df.Cleaned.apply(func = make_lower_case)
df['Cleaned'] = df.Cleaned.apply(func = remove_stop_words)
df['Cleaned'] = df.Cleaned.apply(func = remove_punctuation)
df['Cleaned'] = df.Cleaned.apply(func = remove_html)
df['Cleaned'] = df.Cleaned.apply(func = lemm_text)
df = df.drop('content' , axis =1)

In [5]:
io = df.copy()

io = io.drop([374]).reset_index(drop=True)
io = io.drop([6527]).reset_index(drop=True)


corpus_full = []
for words in io['Cleaned']:
    corpus_full.append(words.split())  


In [6]:
#word2vec_model = Word2Vec.load("word2vec.model3")
word2vec_model = Word2Vec.load("word2vec.i_dont_even_know_how_shitty_this_model_is")

In [7]:
print(word2vec_model)

Word2Vec<vocab=17960, vector_size=500, alpha=0.025>


In [8]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

## Tokenizing our entry points , more to be announced

In [9]:
token = Tokenizer(17960)

In [10]:
token.fit_on_texts(io['Cleaned'])

In [11]:
text = token.texts_to_sequences(io['Cleaned'])

In [12]:
text = pad_sequences(text, 75)
print(text[:2])

[[  105     3    94   735   100  4959  1655  1124   215 10304  6481     2
    976 16165  3926    45    60   830   343  3319   910  2845     3    94
   8434     7  1655 11892 16166 12409     2   236  3926     1  9443  8864
    100  8058     2   236  4475   140   130  1470    24  2537   229   263
    329  8434   211   735  1883   697 10852  7352   385   143    18   236
    100   725   164   166  3288  1655    64  1717  7122    24  7901  1593
   1594 12409  1883]
 [   75  4866  1026 13327  2869  4138  5990  1655  1249 14382 10015  2855
    215   697   105   174   578  2998   528   411   241  1298  1231  9092
    211   751   963  1825  6998   766  8158 14383   432    76   313     1
   1534  2744  5705    64    18   217  1162   400 15231  8866  5129  1316
    661   308     1    45    76  2869   139     7  1984 10169  2863  5008
      2  2734  6574   485  1711   776     1  1128   511   895 14384  1154
   3637  2644    33]]


In [13]:
word_vectors= word2vec_model.wv
for i in range (0,3):
    print(f"{i}: '{word2vec_model.wv.index_to_key[i]}'")

0: 'said'
1: 's'
2: 'year'


In [14]:
vocabular1 = word2vec_model.wv.index_to_key
token1zer = Tokenizer(17960)
token1zer.fit_on_texts(vocabular1)

In [15]:
text2 = token1zer.texts_to_sequences(io['Cleaned'])

In [16]:
text2 = pad_sequences(text2, 75)
print(text2[:2])

[[  105     3    94   734   100  4984  1655  1124   215 10447  6481     2
    976 16168  3932    45    60   834   343  3328   910  2845     3    94
   8516     7  1655 11966 16162 12495     2   236  3932     1  9467  8870
    100  8122     2   236  4492   140   130  1470    24  2540   230   264
    329  8516   211   734  1888   697 10898  7423   385   143    18   236
    100   725   164   166  3292  1655    64  1716  7165    24  7966  1595
   1594 12495  1888]
 [   75  4882  1027 13412  2876  4149  6004  1655  1254 14541 10107  2856
    215   697   105   174   579  3000   528   412   241  1297  1231  9146
    211   752   964  1825  7041   766  8237 14528   432    76   312     1
   1535  2743  5732    64    18   217  1163   400 15557  8917  5133  1317
    663   308     1    45    76  2876   139     7  1986 10253  2868  5032
      2  2735  6601   487  1717   776     1  1128   512   895 14505  1157
   3649  2649    33]]


In [17]:
# Now every word is tokenized gamw to feleki mou

In [18]:
le =LabelEncoder()
y = le.fit_transform(io["category_level_1"])

In [19]:
y

array([2, 2, 2, ..., 1, 1, 1])

In [20]:
y_cat = to_categorical(y)

In [21]:
y_cat

array([[0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]], dtype=float32)

In [22]:
X_train, X_test, y_train, y_test = train_test_split(np.array(text2), y, test_size=0.2 , random_state = 42)

In [23]:
y_train_cat = to_categorical(y_train)
y_test_cat = to_categorical(y_test)

In [24]:
def gensim_to_keras_embedding(model, train_embeddings=False):
    """Get a Keras 'Embedding' layer with weights set from Word2Vec model's learned word embeddings.

    Parameters
    ----------
    train_embeddings : bool
        If False, the returned weights are frozen and stopped from being updated.
        If True, the weights can / will be further updated in Keras.

    Returns
    -------
    `keras.layers.Embedding`
        Embedding layer, to be used as input to deeper network layers.

    """
    keyed_vectors = model.wv  # structure holding the result of training
    weights = keyed_vectors.vectors  # vectors themselves, a 2D numpy array    
    index_to_key = keyed_vectors.index_to_key  # which row in `weights` corresponds to which word?

    layer = Embedding(
        input_dim=weights.shape[0],
        output_dim=weights.shape[1],
        weights=[weights],
        trainable=train_embeddings,
    )
    return layer

In [25]:
emb_layer = gensim_to_keras_embedding(word2vec_model)

In [26]:
cnn_model = Sequential()
cnn_model.add(tf.keras.layers.Input(75,1))  # tokenized X Input shape!
cnn_model.add(emb_layer)
cnn_model.add(tf.keras.layers.Conv1D(128,2, activation='leaky_relu')) # for the first comment 256 == 128
#cnn_model.add(tf.keras.layers.MaxPool1D())
cnn_model.add(tf.keras.layers.Conv1D(128,2,activation='leaky_relu')) #for the first comment 128 == 64
#cnn_model.add(tf.keras.layers.MaxPool1D())
cnn_model.add(tf.keras.layers.Conv1D(128,2,activation='leaky_relu')) 
cnn_model.add(tf.keras.layers.MaxPool1D())
cnn_model.add(tf.keras.layers.Conv1D(64,2,activation='leaky_relu'))
cnn_model.add(tf.keras.layers.Conv1D(64,2,activation='leaky_relu'))
cnn_model.add(tf.keras.layers.Conv1D(64,2,activation='leaky_relu'))
cnn_model.add(tf.keras.layers.MaxPool1D())
cnn_model.add(tf.keras.layers.BatchNormalization())
cnn_model.add(tf.keras.layers.Flatten())
cnn_model.add(tf.keras.layers.Dense(64,activation='leaky_relu'))
cnn_model.add(tf.keras.layers.Dense(32,activation='leaky_relu'))
cnn_model.add(tf.keras.layers.Dense(17,activation='softmax'))

In [27]:
opt = tf.keras.optimizers.AdamW(learning_rate =0.007 , beta_1=0.9,
    beta_2=0.999,
    use_ema=True,
    ema_momentum=0.99)

In [28]:
cnn_model.compile(optimizer=opt, 
          loss = 'categorical_crossentropy',
          metrics=[tf.keras.metrics.BinaryAccuracy(name='accuracy'),
                   tf.keras.metrics.Precision(name='precision') ,
                   tf.keras.metrics.Recall(name='recall')])

In [29]:
cnn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (1, 75, 500)              8980000   
                                                                 
 conv1d (Conv1D)             (1, 74, 128)              128128    
                                                                 
 conv1d_1 (Conv1D)           (1, 73, 128)              32896     
                                                                 
 conv1d_2 (Conv1D)           (1, 72, 128)              32896     
                                                                 
 max_pooling1d (MaxPooling1  (1, 36, 128)              0         
 D)                                                              
                                                                 
 conv1d_3 (Conv1D)           (1, 35, 64)               16448     
                                                        

In [30]:
cnn_model.fit(X_train, 
              y_train_cat , 
              batch_size = 32 , 
              epochs = 1 , 
              validation_split = (0.2) ,
              #lass_weight=class_weights ,
              verbose =1 
             )


219/219 [==============================] - 12s 33ms/step - loss: 2.6718 - accuracy: 0.9416 - precision: 0.5580 - recall: 0.0331 - val_loss: 2.8004 - val_accuracy: 0.9405 - val_precision: 0.4667 - val_recall: 0.0761


In [31]:
predictions = np.argmax(cnn_model.predict(X_test), axis=-1)

69/69 [==============================] - 1s 11ms/step


In [32]:
predictions

array([ 7, 14,  3, ...,  7,  7,  7], dtype=int64)

In [33]:
from sklearn.metrics import classification_report, confusion_matrix

In [34]:
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        68
           1       1.00      0.01      0.01       176
           2       0.00      0.00      0.00        97
           3       0.00      0.00      0.00        90
           4       0.00      0.00      0.00        93
           5       0.00      0.00      0.00       108
           6       0.00      0.00      0.00       126
           7       0.07      0.99      0.13       136
           8       0.00      0.00      0.00       122
           9       0.00      0.00      0.00       155
          10       0.00      0.00      0.00        61
          11       0.00      0.00      0.00       172
          12       0.29      0.05      0.08       182
          13       0.00      0.00      0.00       151
          14       0.03      0.02      0.02       200
          15       0.00      0.00      0.00       169
          16       1.00      0.05      0.10        77

    accuracy              

C:\Users\Desktop\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Desktop\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Desktop\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [35]:
## Lets run it 8 more times

In [36]:
cnn_model.fit(X_train, 
              y_train_cat , 
              batch_size = 32 , 
              epochs = 8 , 
              validation_split = (0.2) ,
              #lass_weight=class_weights ,
              verbose =1 
             )

Epoch 1/8
219/219 [==============================] - 6s 28ms/step - loss: 2.3516 - accuracy: 0.9436 - precision: 0.6740 - recall: 0.0793 - val_loss: 3.5850 - val_accuracy: 0.9230 - val_precision: 0.2239 - val_recall: 0.1254
Epoch 2/8
219/219 [==============================] - 6s 30ms/step - loss: 2.1254 - accuracy: 0.9451 - precision: 0.7012 - recall: 0.1162 - val_loss: 2.5323 - val_accuracy: 0.9400 - val_precision: 0.4643 - val_recall: 0.1339
Epoch 3/8
219/219 [==============================] - 7s 32ms/step - loss: 1.9341 - accuracy: 0.9468 - precision: 0.7061 - recall: 0.1631 - val_loss: 2.1095 - val_accuracy: 0.9443 - val_precision: 0.5695 - val_recall: 0.2158
Epoch 4/8
219/219 [==============================] - 7s 31ms/step - loss: 1.7880 - accuracy: 0.9486 - precision: 0.7143 - recall: 0.2105 - val_loss: 2.5103 - val_accuracy: 0.9390 - val_precision: 0.4565 - val_recall: 0.1952
Epoch 5/8
219/219 [==============================] - 6s 29ms/step - loss: 1.6601 - accuracy: 0.9509 - pr

In [37]:
predictions = np.argmax(cnn_model.predict(X_test), axis=-1)

69/69 [==============================] - 1s 11ms/step


In [38]:
predictions

array([ 9,  1, 16, ..., 14,  2, 12], dtype=int64)

In [39]:
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.38      0.09      0.14        68
           1       0.72      0.49      0.58       176
           2       0.33      0.20      0.25        97
           3       0.43      0.46      0.44        90
           4       0.34      0.12      0.18        93
           5       0.54      0.51      0.52       108
           6       0.68      0.58      0.62       126
           7       0.40      0.49      0.44       136
           8       0.29      0.33      0.31       122
           9       0.57      0.53      0.55       155
          10       0.43      0.16      0.24        61
          11       0.30      0.38      0.33       172
          12       0.57      0.55      0.56       182
          13       0.44      0.30      0.35       151
          14       0.20      0.45      0.27       200
          15       0.79      0.72      0.75       169
          16       0.82      0.78      0.80        77

    accuracy              

In [40]:
# and another 11 would b nice

In [41]:
cnn_model.fit(X_train, 
              y_train_cat , 
              batch_size = 32 , 
              epochs = 6 , 
              validation_split = (0.2) ,
              #lass_weight=class_weights ,
              verbose =1 
             )

Epoch 1/6
219/219 [==============================] - 7s 34ms/step - loss: 1.1009 - accuracy: 0.9623 - precision: 0.7710 - recall: 0.5099 - val_loss: 3.1260 - val_accuracy: 0.9380 - val_precision: 0.4618 - val_recall: 0.3291
Epoch 2/6
219/219 [==============================] - 7s 30ms/step - loss: 0.9895 - accuracy: 0.9649 - precision: 0.7823 - recall: 0.5578 - val_loss: 2.8733 - val_accuracy: 0.9364 - val_precision: 0.4446 - val_recall: 0.3263
Epoch 3/6
219/219 [==============================] - 7s 30ms/step - loss: 0.9033 - accuracy: 0.9678 - precision: 0.7941 - recall: 0.6117 - val_loss: 3.6561 - val_accuracy: 0.9342 - val_precision: 0.4238 - val_recall: 0.3309
Epoch 4/6
219/219 [==============================] - 7s 30ms/step - loss: 0.7963 - accuracy: 0.9719 - precision: 0.8194 - recall: 0.6691 - val_loss: 3.1631 - val_accuracy: 0.9405 - val_precision: 0.4927 - val_recall: 0.3875
Epoch 5/6
219/219 [==============================] - 7s 30ms/step - loss: 0.6996 - accuracy: 0.9748 - pr

In [42]:
predictions = np.argmax(cnn_model.predict(X_test), axis=-1)
predictions

69/69 [==============================] - 1s 10ms/step


array([14,  1, 16, ..., 11, 14, 12], dtype=int64)

In [43]:
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.22      0.12      0.15        68
           1       0.68      0.57      0.62       176
           2       0.42      0.46      0.44        97
           3       0.39      0.48      0.43        90
           4       0.35      0.25      0.29        93
           5       0.64      0.58      0.61       108
           6       0.66      0.50      0.57       126
           7       0.45      0.51      0.48       136
           8       0.38      0.34      0.35       122
           9       0.62      0.56      0.59       155
          10       0.52      0.28      0.36        61
          11       0.32      0.38      0.35       172
          12       0.60      0.49      0.54       182
          13       0.43      0.40      0.41       151
          14       0.26      0.42      0.32       200
          15       0.74      0.75      0.74       169
          16       0.74      0.83      0.78        77

    accuracy              

In [168]:
cnn_model.fit(X_train, 
              y_train_cat , 
              batch_size = 32 , 
              epochs = 5 , 
              validation_split = (0.2) ,
              #lass_weight=class_weights ,
              verbose =1 
             )

predictions = np.argmax(cnn_model.predict(X_test), axis=-1)
print(predictions)

print(classification_report(y_test, predictions))

Epoch 1/5
219/219 [==============================] - 7s 34ms/step - loss: 0.2620 - accuracy: 0.9920 - precision: 0.9514 - recall: 0.9108 - val_loss: 4.0628 - val_accuracy: 0.9325 - val_precision: 0.4146 - val_recall: 0.3600
Epoch 2/5
219/219 [==============================] - 7s 33ms/step - loss: 0.2915 - accuracy: 0.9909 - precision: 0.9422 - recall: 0.9004 - val_loss: 3.6961 - val_accuracy: 0.9350 - val_precision: 0.4393 - val_recall: 0.3812
Epoch 3/5
219/219 [==============================] - 7s 33ms/step - loss: 0.2683 - accuracy: 0.9913 - precision: 0.9446 - recall: 0.9054 - val_loss: 4.8513 - val_accuracy: 0.9264 - val_precision: 0.3609 - val_recall: 0.3268
Epoch 4/5
219/219 [==============================] - 7s 32ms/step - loss: 0.2867 - accuracy: 0.9910 - precision: 0.9420 - recall: 0.9024 - val_loss: 4.1348 - val_accuracy: 0.9340 - val_precision: 0.4288 - val_recall: 0.3686
Epoch 5/5
69/69 [==============================] - 1s 11ms/step
[ 9  1 16 ... 12  5  1]
              pr

## Okay time for model2

In [49]:
cnn_model2 = Sequential()
cnn_model2.add(tf.keras.layers.Input(75,1))  # tokenized X Input shape!
cnn_model2.add(emb_layer)
cnn_model2.add(tf.keras.layers.Conv1D(256,2,padding='same', strides=1 ,activation='relu')) 
cnn_model2.add(tf.keras.layers.Conv1D(128,2,padding='same', strides=1 ,activation='relu')) 
cnn_model2.add(tf.keras.layers.Conv1D(64,2,padding='same', strides=1 ,activation='relu')) 
cnn_model2.add(tf.keras.layers.MaxPool1D())
#cnn_model2.add(tf.keras.layers.BatchNormalization())
cnn_model2.add(tf.keras.layers.Flatten())
cnn_model2.add(tf.keras.layers.Dense(64,activation='leaky_relu'))
cnn_model2.add((tf.keras.layers.Dropout(0.2)))
#cnn_model2.add(tf.keras.layers.Dense(32,activation='leaky_relu'))
cnn_model2.add(tf.keras.layers.Dense(17,activation='softmax'))

In [50]:
opt = tf.keras.optimizers.AdamW(learning_rate =0.0005 , beta_1=0.9,
    beta_2=0.999,
    use_ema=True,
    ema_momentum=0.99)

In [51]:
cnn_model2.compile(optimizer=opt, 
          loss = 'categorical_crossentropy',
          metrics=[tf.keras.metrics.BinaryAccuracy(name='accuracy'),
                   tf.keras.metrics.Precision(name='precision') ,
                   tf.keras.metrics.Recall(name='recall')])

In [52]:
cnn_model2.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (1, 75, 500)              8980000   
                                                                 
 conv1d_9 (Conv1D)           (1, 75, 256)              256256    
                                                                 
 conv1d_10 (Conv1D)          (1, 75, 128)              65664     
                                                                 
 conv1d_11 (Conv1D)          (1, 75, 64)               16448     
                                                                 
 max_pooling1d_3 (MaxPoolin  (1, 37, 64)               0         
 g1D)                                                            
                                                                 
 flatten_2 (Flatten)         (1, 2368)                 0         
                                                      

In [53]:
cnn_model2.fit(X_train, 
              y_train_cat , 
              batch_size = 64 , 
              epochs = 10 , 
              validation_split = (0.2) ,
              #lass_weight=class_weights ,
              verbose =1 
             )

predictions = np.argmax(cnn_model2.predict(X_test), axis=-1)
print(predictions)

print(classification_report(y_test, predictions))

Epoch 1/10
110/110 [==============================] - 11s 79ms/step - loss: 2.7719 - accuracy: 0.9412 - precision: 1.0000 - recall: 2.8633e-04 - val_loss: 2.6515 - val_accuracy: 0.9413 - val_precision: 1.0000 - val_recall: 0.0023
Epoch 2/10
110/110 [==============================] - 8s 77ms/step - loss: 2.4699 - accuracy: 0.9428 - precision: 0.8403 - recall: 0.0346 - val_loss: 2.3190 - val_accuracy: 0.9454 - val_precision: 0.8000 - val_recall: 0.0962
Epoch 3/10
110/110 [==============================] - 8s 76ms/step - loss: 2.0402 - accuracy: 0.9459 - precision: 0.8103 - recall: 0.1058 - val_loss: 2.1158 - val_accuracy: 0.9452 - val_precision: 0.7990 - val_recall: 0.0910
Epoch 4/10
110/110 [==============================] - 8s 75ms/step - loss: 1.6157 - accuracy: 0.9508 - precision: 0.8354 - recall: 0.2027 - val_loss: 2.1075 - val_accuracy: 0.9450 - val_precision: 0.6342 - val_recall: 0.1528
Epoch 5/10
110/110 [==============================] - 9s 78ms/step - loss: 1.2290 - accuracy: 0

In [120]:
# another 10 maybe

In [121]:
cnn_model2.fit(X_train, 
              y_train_cat , 
              batch_size = 32 , 
              epochs = 10 , 
              validation_split = (0.2) ,
              #lass_weight=class_weights ,
              verbose =1 
             )

predictions = np.argmax(cnn_model2.predict(X_test), axis=-1)
print(predictions)

print(classification_report(y_test, predictions))

Epoch 1/10
219/219 [==============================] - 10s 47ms/step - loss: 0.0154 - accuracy: 0.9969 - precision: 0.9783 - recall: 0.9692 - val_loss: 0.5833 - val_accuracy: 0.9044 - val_precision: 0.1579 - val_recall: 0.1442
Epoch 2/10
219/219 [==============================] - 10s 45ms/step - loss: 0.0152 - accuracy: 0.9971 - precision: 0.9798 - recall: 0.9711 - val_loss: 0.3663 - val_accuracy: 0.9218 - val_precision: 0.2962 - val_recall: 0.2393
Epoch 3/10
219/219 [==============================] - 10s 47ms/step - loss: 0.0141 - accuracy: 0.9973 - precision: 0.9814 - recall: 0.9727 - val_loss: 0.5427 - val_accuracy: 0.9084 - val_precision: 0.2010 - val_recall: 0.1872
Epoch 4/10
219/219 [==============================] - 10s 46ms/step - loss: 0.0136 - accuracy: 0.9972 - precision: 0.9798 - recall: 0.9731 - val_loss: 0.5824 - val_accuracy: 0.9037 - val_precision: 0.1683 - val_recall: 0.1614
Epoch 5/10
219/219 [==============================] - 10s 45ms/step - loss: 0.0130 - accuracy: 0